In [33]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, LinearRegression
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer

In [41]:
def do_pca(testfilepath,trainfilepath,predictfilepath):
    data_test=pd.read_csv(testfilepath,index_col=False)
    data_train=pd.read_csv(trainfilepath,index_col=False)
    data_pred=pd.read_csv(predictfilepath,index_col=False)
    data_pred=data_pred[data_pred.RegionName.apply(lambda x:x[-7:][:4])=='2019']
    data_trainx=data_train.iloc[:,4:]
    data_testx=data_test.iloc[:,4:]
    data_predx=data_pred.iloc[:,4:]
    train_y=data_train.iloc[:,3]
    test_y=data_test.iloc[:,3]
    pred_y=data_pred.iloc[:,3]
    pred_y=np.log(pred_y)
    test_y=np.log(test_y)
    train_y=np.log(train_y)
    #fit pca w max components 
    # pca (stands for perfect charming angel <3)
    pca = PCA(n_components=10,random_state=100)
    pca.fit(data_trainx)

    #how many components to explain 95% of variance 
    i=0
    cum_exp_var=0
    while cum_exp_var<.95:

        cum_exp_var+=pca.explained_variance_ratio_[i]
        i+=1


    #refit pca w # components that explain 95% variance 
    pca = PCA(n_components=i,random_state=100)
    pca.fit(data_trainx)
    pca_train=pca.transform(data_trainx)
    pca_train=pd.DataFrame(data=pca_train)
    pca_test=pca.transform(data_testx)
    pca_test=pd.DataFrame(data=pca_test)
    pca_pred=pca.transform(data_predx)
    pca_pred=pd.DataFrame(data=pca_pred)
    
    #add back logged y vals
    pca_pred['Log_Rent']=pred_y
    pca_test['Log_Rent']=test_y  
    pca_train['Log_Rent']=train_y

        
    return [pca_test,pca_train,pca_pred]


In [38]:
##JULY 17

#pca on full acs and our new hospitals gov buildings businesses data
[pca_test,pca_train,pca_pred]=do_pca('TESTDATAnewcols7.17_3yearPCA.csv','TRAINDATAnewcols7.17_3yearPCA.csv','PREDICTDATAnewcols7.17_3yearPCA.csv')
pca_test= pd.DataFrame(pca_test)
pca_train= pd.DataFrame(pca_train)
pca_pred= pd.DataFrame(pca_pred)
pca_test.to_csv('Results/TESTDATAnewcols7.17_3yearPCAfull.csv',index=False)
pca_train.to_csv('Results/TRAINDATAnewcols7.17_3yearPCAfull.csv',index=False)
pca_pred.to_csv('Results/PREDICTDATAnewcols7.17_3yearPCAfull.csv',index=False)


#pca on selected/modified acs and our new hospitals gov buildings businesses data
[pca_test,pca_train,pca_pred]=do_pca('TESTDATAnewcols7.17_3year.csv','TRAINDATAnewcols7.17_3year.csv','PREDICTDATAnewcols7.17_3year.csv')

pca_test.to_csv('Results/TESTDATAnewcols7.17_3yearPCApartial.csv',index=False)
pca_train.to_csv('Results/TRAINDATAnewcols7.17_3yearPCApartial.csv',index=False)
pca_pred.to_csv('Results/PREDICTDATAnewcols7.17_3yearPCApartial.csv',index=False)




In [45]:
## JULY 19

#pca on selected/modified acs and our new hospitals gov buildings businesses data
[pca_test,pca_train,pca_pred]=do_pca('TESTDATAnewcols7.18_3year.csv','TRAINDATAnewcols7.18_3year.csv','PREDICTDATAnewcols7.18_3year.csv')


pca_test.to_csv('Results/TESTDATAnewcols7.18_3yearPCApartial.csv',index=False)
pca_train.to_csv('Results/TRAINDATAnewcols7.18_3yearPCApartial.csv',index=False)
pca_pred.to_csv('Results/PREDICTDATAnewcols7.18_3yearPCApartial.csv',index=False)






In [30]:
# #lasso on pca

# std_scaler=StandardScaler()
# train_X_scaled = std_scaler.fit_transform(pca_train)
# test_X_scaled = std_scaler.transform(pca_test)
# pred_X_scaled=std_scaler.transform(pca_pred)

# #copied from ryans lasso code
# # Create and fit the model
# lasso = Lasso(normalize=False)
# lasso_params = {'alpha':np.linspace(0,10,100)}
# grid_lasso = GridSearchCV(lasso, lasso_params, scoring='r2', n_jobs=-1, cv=5, verbose=2)
# grid_lasso.fit(train_X_scaled, train_y)
    
# # Tell user alpha used
# print('Best alpha: ', grid_lasso.best_params_['alpha'])
    
# # Tell user training score
# print('Score on training data: ', round(grid_lasso.score(train_X_scaled,train_y),3))

    
# # Tell user the croo-validation scores
# print('Scores from 5-fold cross-validation: \n', cross_val_score(grid_lasso, train_X_scaled, train_y, scoring='r2', n_jobs=-1, cv=5))
    
# # Tell user the test score
# print('Score on TEST data: ', round(grid_lasso.score(test_X_scaled,test_y),3))
    
    
# # Tell user pred score
# print('Score on pred data: ', round(grid_lasso.score(pred_X_scaled,pred_y),3))

In [31]:

# Scoretest=[]
# Scoretrain=[]
# Scorepred=[]
# alphrange=np.linspace(0,10,100)
# for i in alphrange:
       
#     lasso = Lasso(normalize=False,alpha=i)
#     lasso.fit(train_X_scaled, train_y)
#     Scoretest.append(lasso.score(test_X_scaled,test_y))
#     Scoretrain.append(lasso.score(train_X_scaled,train_y))
#     Scorepred.append(lasso.score(pred_X_scaled,pred_y))
# print('best alpha for test r2 is ',alphrange[Scoretest.index(max(Scoretest))])
# print('train and test ',Scoretrain[Scoretest.index(max(Scoretest))],Scoretest[Scoretest.index(max(Scoretest))],Scorepred[Scoretest.index(max(Scoretest))])

In [32]:
##pca on our subsect of data 30 ish cols 